<a href="https://colab.research.google.com/github/kleberquaresma/alura_imersao_ia_google/blob/main/projeto_final_imersao_ia_alura_google.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

⚙

**Instalando a biblioteca do Google para uso da IA Generativa (Gemini):**

OBS.: *Para execução em um terminal local, fora do ambiente Colab, se faz necessário antes instalar o Python (https://www.python.org/downloads/) e certificar-se que o caminho da instalação dele está definido nas variáveis de ambiente do sistema operacional.*

In [1]:
!pip install -q -U google-generativeai


📥

**Importando as bibliotecas necessárias à execução do código.**

In [12]:
#-------------------------------------------------------------------------
# INÍCIO das Importações necessárias (bibliotecas, módulos, funções....)
#-------------------------------------------------------------------------

# Importação da biblioteca para melhor trabalhar com cainhos de arquivo.
import pathlib

# Importação da biblioteca para formatação de blocos de texto.
import textwrap

# Importação da biblioteca SDK para uso dos modelos de IA do Gemini.
import google.generativeai as genai

# Importação de função que possibilita exibir resultados de forma mais amigável (tabelas, imagens, etc).
# Importação de classe que possibilita exibir textos utilizando a formação MarkDown.
from IPython.display import display, Markdown

# Importação de função que possibilita acessar dados, como a Chave da API, de forma segura.
from google.colab import userdata

#-------------------------------------------------------------------------
# FIM das Importações necessárias (bibliotecas, módulos, funções....)
#-------------------------------------------------------------------------

# Função que permitirá formatar o texto em MarkDown realizando algumas tratativas.
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

🔑

**Adicionando a chave da API que será utilizada para realização da integração com o ambiente do Gemini da Google:**

Obs.: *Necessário antes inserir a chave da API, previamente criada através do Google AI Studio, no gerenciador de chaves do Colab. Caso contrário inserir a chave em substituição ao texto 'GOOGLE_API_KEY'.*

In [5]:
# Or use `os.getenv('GOOGLE_API_KEY')` to fetch an environment variable.
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

♊

**Definindo as variáveis do Gemini:**

In [27]:
parametros_gemini = {
    "candidate_count": 1,
    "temperature": 1,
    "top_p": 1,
    "top_k": 1,
}

from google.generativeai.types import HarmCategory, HarmBlockThreshold

seguranca_gemini = {
    'HARASSMENT': 'block_none',
    'SEXUAL': 'block_none',
    'DANGEROUS': 'block_none',
    'HATE': 'block_none',
}

♊

**Definindo o modelo de IA do Gemini a ser utilizado:**

OBS.: *Para interação multimodal (texto, imagens e vídeo) utilizar um dos modelos "VISIO".*

In [28]:
# Lista os modelos disponíveis:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

# Define o modelo a ser utilizado:
model = genai.GenerativeModel('gemini-pro-vision',
                              safety_settings=seguranca_gemini,
                              generation_config=parametros_gemini)

modelo_escolhido = Markdown('**Modelo Escolhido:**')
display(modelo_escolhido, model.model_name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


**Modelo Escolhido:**

'models/gemini-pro-vision'